In [1]:
import os

# 현재 디렉터리에 'data' 디렉터리가 있는지 확인
if os.path.exists("data") and os.path.isdir("data"):
    data_path = "data/"
else:
    # 상위 디렉터리 '../data'를 확인
    data_path = "../data/"


In [3]:
from bs4 import BeautifulSoup
import requests
import re  # 정규 표현식 모듈 추가
import pandas as pd



In [57]:
def get_gamefaqs_script(path):
    url = f"https://gamefaqs.gamespot.com/{path}/stats"
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
        # "Accept": "application/json",
        # "X-Requested-With": "XMLHttpRequest",
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    scripts = soup.select("script")
    script_dict = dict()
    for s in scripts:
        if re.search("#stats_pie_diff", s.text):
            script_dict['difficulty'] = s.text
        if re.search("#stats_pie_time", s.text):
            script_dict['playtime'] = s.text
        if re.search("#stats_pie_play", s.text):
            script_dict['achievement'] = s.text
    return script_dict


script_dict = get_gamefaqs_script("iphone/330536-cookie-run-kingdom")
# script_dict = get_gamefaqs_script("pc/954437-league-of-legends")
# script_dict


In [45]:

# pattern = r"var data = .*?];"  # "var data ="로 시작해 "];"로 끝나는 패턴
# match = re.search(pattern, script_dict['difficulty'], re.DOTALL)  # DOTALL 옵션으로 여러 줄 매칭 허용
# idx_s, idx_e = match.span()[0] + 11, match.span()[1] - 1
# script_dict['difficulty'][idx_s:idx_e]


'[{ label:"Just Right - 80%", data: 4, color:"#00d100" },{ label:"Tough - 20%", data: 1, color:"#7f8000" },]'

In [58]:
def parse_stats(script_dict):
    p1 = r"var data = .*?];"
    p2 = r'label:"(.*?)".*?data:\s*(\d+)'

    result_dict = {
        "Simple": 0, "Easy": 0, "Just Right": 0, "Tough": 0, "Unforgiving": 0,  # 난이도
        "&lt; 1 Hour": 0, "~1 Hour": 0, "~2 Hours": 0, "~4 Hours": 0, "~8 Hours": 0, "~12 Hours": 0, "~20 Hours": 0, "~40 Hours": 0, "~60 Hours": 0, "&gt;= 80 Hours": 0,  # 플레이시간
        "Tried It": 0, "Played It": 0, "Halfway": 0, "Beat It": 0, "Conquered It": 0, # 성취도
        }

    for sd in script_dict:
        s_tmp = script_dict[sd]
        match = re.search(p1, s_tmp, re.DOTALL)  # DOTALL 옵션으로 여러 줄 매칭 허용
        idx_s, idx_e = match.span()[0] + 11, match.span()[1] - 1
        d_tmp = s_tmp[idx_s:idx_e]
        matches = re.findall(p2, d_tmp)
        result_dict.update({label.split(" - ")[0]: int(data) for label, data in matches})

    return result_dict

parse_stats(script_dict)

{'Simple': 0,
 'Easy': 0,
 'Just Right': 4,
 'Tough': 1,
 'Unforgiving': 0,
 '&lt; 1 Hour': 0,
 '~1 Hour': 0,
 '~2 Hours': 0,
 '~4 Hours': 0,
 '~8 Hours': 0,
 '~12 Hours': 0,
 '~20 Hours': 0,
 '~40 Hours': 0,
 '~60 Hours': 0,
 '&gt;= 80 Hours': 5,
 'Tried It': 0,
 'Played It': 1,
 'Halfway': 3,
 'Beat It': 0,
 'Conquered It': 1}

In [59]:
# gamefaqs 데이터 수집 진행
gamefaqs_game_list = [
    ("쿠키런킹덤", 'android/332328-cookie-run-kingdom'),
    ("쿠키런킹덤", "iphone/330536-cookie-run-kingdom"),
    ("던전앤파이터", "pc/937883-dungeon-fighter-online"),
    ("메이플스토리", "pc/924697-maplestory"),
    ("월드오브워크래프트", "pc/534914-world-of-warcraft"),
    ("블레이드앤소울", "pc/955631-blade-and-soul"),
    # 마비노기
    ("리그오브레전드", "pc/954437-league-of-legends"),
    # 펜타스톰
    ("도타2", "pc/610657-dota-2"),
    ("브롤스타즈", "android/254094-brawl-stars"),
    ("브롤스타즈", "iphone/253930-brawl-stars"),
    ("서든어택", "pc/960351-sudden-attack"),
    ("배틀그라운드", "pc/206545-playerunknowns-battlegrounds"),
    ("배틀그라운드", 'android/235332-pubg-mobile'),
    ("오버워치", "ps4/182930-overwatch"),
    ("발로란트", "pc/285009-valorant"),
    ("클래시오브클랜", "iphone/678942-clash-of-clans"),
    ("클래시로얄", "android/188972-clash-royale"),
    ("스타크래프트", "pc/25418-starcraft"),
    # 피파온라인
    ("피파23", "ps5/373098-fifa-23"),
    ("캔디크러쉬사가", "android/702058-candy-crush-saga"),
    ("마인크래프트", "pc/606524-minecraft"),
    ("로블록스", "pc/185091-roblox"),
    ("카트라이더", "pc/939366-kartrider"),
    ("테일즈런너", "pc/952467-tales-runner"),
    # 크레이지아케이드
    ("모여봐요동물의숲", "switch/248082-animal-crossing-new-horizons"),
    # 모두의마블
    ("포켓몬스터", "switch/359435-pokemon-scarlet"),
    # ("마비노기", "pc/926429-mabinogi"),
    # ("아레나오브발러", "switch/221219-arena-of-valor"),
    # ("NBA2K", "ps5/394854-mlb-the-show-23"),
]


stat_data = []
for name, path in gamefaqs_game_list:
    game_stat = {"name": name}
    script_dict = get_gamefaqs_script(path)
    game_stat.update(parse_stats(script_dict))
    print(game_stat)
    stat_data.append(game_stat)
    


{'name': '쿠키런킹덤', 'Simple': 1, 'Easy': 0, 'Just Right': 5, 'Tough': 0, 'Unforgiving': 1, '&lt; 1 Hour': 1, '~1 Hour': 0, '~2 Hours': 0, '~4 Hours': 0, '~8 Hours': 0, '~12 Hours': 0, '~20 Hours': 0, '~40 Hours': 0, '~60 Hours': 0, '&gt;= 80 Hours': 4, 'Tried It': 2, 'Played It': 0, 'Halfway': 4, 'Beat It': 1, 'Conquered It': 1}
{'name': '쿠키런킹덤', 'Simple': 0, 'Easy': 0, 'Just Right': 4, 'Tough': 1, 'Unforgiving': 0, '&lt; 1 Hour': 0, '~1 Hour': 0, '~2 Hours': 0, '~4 Hours': 0, '~8 Hours': 0, '~12 Hours': 0, '~20 Hours': 0, '~40 Hours': 0, '~60 Hours': 0, '&gt;= 80 Hours': 5, 'Tried It': 0, 'Played It': 1, 'Halfway': 3, 'Beat It': 0, 'Conquered It': 1}
{'name': '던전앤파이터', 'Simple': 3, 'Easy': 6, 'Just Right': 52, 'Tough': 14, 'Unforgiving': 3, '&lt; 1 Hour': 1, '~1 Hour': 0, '~2 Hours': 0, '~4 Hours': 1, '~8 Hours': 1, '~12 Hours': 0, '~20 Hours': 2, '~40 Hours': 1, '~60 Hours': 5, '&gt;= 80 Hours': 23, 'Tried It': 43, 'Played It': 88, 'Halfway': 21, 'Beat It': 22, 'Conquered It': 9}
{'nam

In [93]:
df_stat_data = pd.DataFrame(stat_data)
df_stat_data = df_stat_data.groupby('name', as_index=False).sum()
# 난이도, 플레이시간, 성취도 점수 생성
difficulty_labels = ["Simple", "Easy", "Just Right", "Tough", "Unforgiving"]  # 난이도
playtime_labels = ["&lt; 1 Hour", "~1 Hour", "~2 Hours", "~4 Hours", "~8 Hours", "~12 Hours", "~20 Hours", "~40 Hours", "~60 Hours", "&gt;= 80 Hours"]  # 플레이시간
achievement_labels = ["Tried It", "Played It", "Halfway", "Beat It", "Conquered It"]  # 성취도

# 점수 계산 함수
def calculate_weighted_score(df, labels):
    total_counts = df[labels].sum(axis=1)  # 각 행의 총합 계산
    return round(df[labels].mul(range(1, len(labels) + 1), axis=1).sum(axis=1) / total_counts, 2)

# 각 라벨에 대해 점수 계산 및 새로운 컬럼 추가
df_stat_data["difficulty_score"] = calculate_weighted_score(df_stat_data, difficulty_labels)
df_stat_data["playtime_score"] = calculate_weighted_score(df_stat_data, playtime_labels)
df_stat_data["achievement_score"] = calculate_weighted_score(df_stat_data, achievement_labels)
print(df_stat_data.loc[:5, ["name"] + difficulty_labels + ["difficulty_score"]])
print(df_stat_data.loc[:5, ["name"] + playtime_labels + ["playtime_score"]])
print(df_stat_data.loc[:5, ["name"] + achievement_labels + ["achievement_score"]])

df_stat_data.columns = [re.sub(r'\W+', '_', col) for col in df_stat_data.columns]  # 컬럼명 특수문자를 '_'로 변경
df_stat_data.to_csv(data_path + "game_stats.csv", index=False)
df_stat_data

      name  Simple  Easy  Just Right  Tough  Unforgiving  difficulty_score
0   던전앤파이터       3     6          52     14            3              3.10
1      도타2      23    26         206    596          328              4.00
2     로블록스      77    78         186     14           16              2.50
3  리그오브레전드     175   128         862   1476          396              3.59
4   마인크래프트     330   784        3317    497           54              2.83
5   메이플스토리      22    58         171     59           10              2.93
      name  &lt; 1 Hour  ~1 Hour  ~2 Hours  ~4 Hours  ~8 Hours  ~12 Hours  \
0   던전앤파이터            1        0         0         1         1          0   
1      도타2           11        6         4         4         7          8   
2     로블록스            7        0         5         7         6         10   
3  리그오브레전드           65       33        11        20        21         23   
4   마인크래프트           36       15        21        33        67         90   
5   메이플스토리   

,name,Simple,Easy,Just_Right,Tough,Unforgiving,_lt_1_Hour,_1_Hour,_2_Hours,_4_Hours,...,_60_Hours,_gt_80_Hours,Tried_It,Played_It,Halfway,Beat_It,Conquered_It,difficulty_score,playtime_score,achievement_score
0,던전앤파이터,3,6,52,14,3,1,0,0,1,...,5,23,43,88,21,22,9,3.10,9.03,2.27
1,도타2,23,26,206,596,328,11,6,4,4,...,10,630,175,478,121,409,283,4.00,9.53,3.10
2,로블록스,77,78,186,14,16,7,0,5,7,...,8,217,16,122,23,60,162,2.50,9.03,3.60
3,리그오브레전드,175,128,862,1476,396,65,33,11,20,...,50,1482,362,1258,376,1182,661,3.59,9.17,3.14
4,마인크래프트,330,784,3317,497,54,36,15,21,33,...,160,2394,661,1803,828,1556,1816,2.83,9.27,3.31
5,메이플스토리,22,58,171,59,10,1,1,0,0,...,2,106,277,661,91,108,67,2.93,9.61,2.19
6,모여봐요동물의숲,577,1256,362,16,19,4,4,6,8,...,50,1541,58,537,365,913,357,1.94,9.69,3.44
7,발로란트,3,2,25,46,13,1,1,2,3,...,8,50,9,40,12,13,14,3.72,8.78,2.81
8,배틀그라운드,5,13,81,145,37,7,1,7,4,...,9,107,27,128,24,87,45,3.70,8.36,2.98
9,브롤스타즈,4,10,40,12,3,1,0,2,1,...,1,33,2,21,18,16,11,3.00,8.72,3.19


### 데이터 분석 및 시각화